In [1]:
import csv
import pandas as pd
import numpy as np
import seaborn as sns
import re
from nltk.corpus import stopwords
import nltk
import string
import matplotlib.pyplot as plt
import random
%matplotlib inline
import os
import json

from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer
from sklearn import decomposition, ensemble
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import cross_val_score

from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
#from nltk.corpus import stopwords
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.multiclass import OneVsRestClassifier

from sklearn.metrics import classification_report
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import confusion_matrix

import joblib

from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
import os,boto3,sys,glob,json

### Part 1  Loading the  data 

#### Read in csv file

In [2]:
testDF=pd.read_csv('run1000charts_0521.csv')
# the csv is generated in D drive D:/chartai_qa  with extract1000charts_jsontxt.py, then uploaded to here

In [3]:
len(testDF)

76792

In [4]:
testDF.columns

Index(['Unnamed: 0', 'chart_id', 'code', 'text', 'start_offset', 'end_offset',
       'text75'],
      dtype='object')

In [5]:
testDF=testDF.drop(['Unnamed: 0'],axis=1)

In [6]:
testDF.head()

,chart_id,code,text,start_offset,end_offset,text75
0,IBXRMA2018_MULT_1163297620010001_HMK_179342168...,E063,Ht,1187,1189,ocedure codes 36415 venipunct routine vital si...
1,IBXRMA2018_MULT_1163297620010001_HMK_179342168...,E785,Hyperlipidemia,1374,1388,tion list reviewed reconciled patient past med...
2,IBXRMA2018_MULT_1163297620010001_HMK_179342168...,I10,Hypertension,1391,1403,reconciled patient past medical history hyperl...
3,IBXRMA2018_MULT_1163297620010001_HMK_179342168...,E785,Hyperlipidemia,2957,2971,tion list reviewed reconciled patient past med...
4,IBXRMA2018_MULT_1163297620010001_HMK_179342168...,I10,Hypertension,2974,2986,reconciled patient past medical history hyperl...


In [7]:
# just check if there's null text, if yes, remove them 
nulltext=testDF[~pd.notnull(testDF['text'])] 
len(nulltext)

0

In [8]:
nulltext=testDF[~pd.notnull(testDF['text75'])] 
len(nulltext)

26

In [9]:
nulltext   # there are some text not correctly extracted

,chart_id,code,text,start_offset,end_offset,text75
1336,IBXRMA2018_MULT_1164353980010001_HMK_160303709...,G129,SMA,4,7,NaN
2198,IBXRMA2018_MULT_1176161080010001_HMK_192365603...,N08,Kidney,46,52,NaN
3876,IBXRMA2018_MULT_1202740400010001_HMK_153425750...,G129,SMA,4,7,NaN
4032,IBXRMA2018_MULT_1202908350010001_HMK_210648786...,E744,PC,25,27,NaN
5591,IBXRMA2018_MULT_1210779380010001_HMK_184462858...,G129,SMA,4,7,NaN
6465,IBXRMA2018_MULT_1212745310010001_HMK_165441851...,N08,Kidney,48,54,NaN
9116,IBXRMA2018_MULT_1216729130010001_HMK_196286386...,N08,Kidney,46,52,NaN
14970,IBXRMA2018_MULT_1216856710010001_HMK_173309591...,G129,SMA,4,7,NaN
18950,IBXRMA2018_MULT_1216874020010001_HMK_143364782...,E744,PC,25,27,NaN
21748,IBXRMA2018_MULT_1216901800010001_HMK_201224196...,G129,SMA,4,7,NaN


In [10]:
testDF=testDF[pd.notnull(testDF['text75'])] 
len(testDF)

76766

In [11]:
testDF['chart_id'].nunique()   # only 980 charts not 1044 charts

980

In [12]:
testDF=testDF.reset_index(drop=True)   # very very import, to reset the index, otherwise the below running the model will not be right, because I use the index as reference for position and matching them
testDF.tail()

,chart_id,code,text,start_offset,end_offset,text75
76761,IBXRMA2018_MULT_1231614110010001_HMK_196526763...,E119,dm2,5902,5905,onciled patient past medical history cholecyst...
76762,IBXRMA2018_MULT_1231614110010001_HMK_196526763...,B182,hepatitis c,6819,6830,lab comp metabolic panel lab glycosolated hemo...
76763,IBXRMA2018_MULT_1231614110010001_HMK_196526763...,E550,vitamin d deficiency,6951,6971,increat random colon cancer screening notes de...
76764,IBXRMA2018_MULT_1231614110010001_HMK_196526763...,E782,hyperlipidemia,7001,7015,declined gi referral vitamin deficiency lab vi...
76765,IBXRMA2018_MULT_1231614110010001_HMK_196526763...,I2510,cad,7144,7147,breast cancer screening imaging mammo digital ...


### Part 2 Loading the tokenizer 

In [13]:
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from keras.models import Sequential
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping

from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import confusion_matrix
from matplotlib import pyplot as plt
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
import pickle

/opt/conda/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [14]:
testDF.columns

Index(['chart_id', 'code', 'text', 'start_offset', 'end_offset', 'text75'], dtype='object')

In [15]:
with open("../../picklefolder_ngrams/temp_4deletionmodels_refreshed/keras_tokenizer_embeddings_0328.pickle", "rb") as f:
   tokenizer = pickle.load(f)    # lOAD tokenizer

In [16]:
max_len = 70

In [17]:
word_index = tokenizer.word_index
word_index

{'disease': 1,
 'hypertension': 2,
 'history': 3,
 'chronic': 4,
 'diabetes': 5,
 'unspecified': 6,
 'hyperlipidemia': 7,
 'type': 8,
 'patient': 9,
 'kidney': 10,
 'mellitus': 11,
 'without': 12,
 'essential': 13,
 'disorder': 14,
 'heart': 15,
 'stable': 16,
 'screening': 17,
 'primary': 18,
 'cancer': 19,
 'plan': 20,
 'family': 21,
 'cholesterol': 22,
 'depression': 23,
 'artery': 24,
 'current': 25,
 'stage': 26,
 'continue': 27,
 'use': 28,
 'pain': 29,
 'atrial': 30,
 'coronary': 31,
 'assessment': 32,
 'benign': 33,
 'medical': 34,
 'left': 35,
 'normal': 36,
 'onset': 37,
 'blood': 38,
 'right': 39,
 'hypercholesterolemia': 40,
 'pulmonary': 41,
 'visit': 42,
 'i10': 43,
 'diabetic': 44,
 'mild': 45,
 'fibrillation': 46,
 'active': 47,
 'osteoporosis': 48,
 'problem': 49,
 'reflux': 50,
 'hypothyroidism': 51,
 'failure': 52,
 'past': 53,
 'anxiety': 54,
 'mixed': 55,
 'care': 56,
 'problems': 57,
 'mg': 58,
 'due': 59,
 'high': 60,
 'htn': 61,
 'reviewed': 62,
 'present': 63,


In [18]:
print(len(testDF))
# tokenize the test dataset
sequences = tokenizer.texts_to_sequences(testDF['text75'].values)
#word_index = tokenizer.word_index
#print('Found %s unique tokens.' % len(word_index))
testX1 = pad_sequences(sequences, maxlen=max_len)
#y_test1 = testdf['flag']
print(testX1.shape)  # ,y_test1.shape)    # no flag for the data

76766
(76766, 70)


### Part 3. Load LSTM model  and apply to test data

In [19]:
# load the lstm model
model2=joblib.load('../../picklefolder_ngrams/temp_4deletionmodels_refreshed/DeleteReason_biclass_Mar2819_lstm.pickle')


W0624 19:29:44.944759 139725060200256 deprecation_wrapper.py:119] From /opt/conda/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0624 19:29:44.960080 139725060200256 deprecation_wrapper.py:119] From /opt/conda/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0624 19:29:44.976010 139725060200256 deprecation_wrapper.py:119] From /opt/conda/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:131: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0624 19:29:44.976789 139725060200256 deprecation_wrapper.py:119] From /opt/conda/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0624 19:29:44.986065

In [20]:
predictiont2 = model2.predict(testX1)

finallabelt1= (predictiont2 > 0.4).astype(np.int)    # using different threshold
#finallabelt2= (predictiont2 > 0.3).astype(np.int) 
#finallabelt3= (predictiont2 > 0.2).astype(np.int)

In [21]:
testDF['pred']=finallabelt1

In [22]:
testDF=testDF.reset_index(drop=True)
testDF.tail()

,chart_id,code,text,start_offset,end_offset,text75,pred
76761,IBXRMA2018_MULT_1231614110010001_HMK_196526763...,E119,dm2,5902,5905,onciled patient past medical history cholecyst...,1
76762,IBXRMA2018_MULT_1231614110010001_HMK_196526763...,B182,hepatitis c,6819,6830,lab comp metabolic panel lab glycosolated hemo...,1
76763,IBXRMA2018_MULT_1231614110010001_HMK_196526763...,E550,vitamin d deficiency,6951,6971,increat random colon cancer screening notes de...,0
76764,IBXRMA2018_MULT_1231614110010001_HMK_196526763...,E782,hyperlipidemia,7001,7015,declined gi referral vitamin deficiency lab vi...,0
76765,IBXRMA2018_MULT_1231614110010001_HMK_196526763...,I2510,cad,7144,7147,breast cancer screening imaging mammo digital ...,1


In [23]:
print(len(set(testDF['chart_id'].tolist())))


980


In [24]:
# to make minimal change in the following code, 
df_pred=testDF

###  Part 4. Load in coder feedback and  evaluate the accuracy

#### Select common chart_ids from both 1000 model run and 6000 coder annotation

In [25]:
# model computation
df_pred1=df_pred[['chart_id', 'code', 'text', 'start_offset', 'end_offset', 'text75','pred']]
print(len(df_pred1))
df_pred1 = df_pred1.drop_duplicates()
print(len(df_pred1))

76766
76766


In [26]:
# read in all charts with coder feedback
df_codeall = pd.read_csv('/home/jovyan/work/Analytics_Data_training/conditions_with_chartids_29042019.csv')
print(len(df_codeall))

136527


In [27]:
print("unique chart is in 1000 charts run model output:",df_pred1['chart_id'].nunique())
print("unique chart is in 6000 charts with coder feedback:",df_codeall['chart_id'].nunique())

unique chart is in 1000 charts run model output: 980
unique chart is in 6000 charts with coder feedback: 5982


In [28]:
# now find overlap chart ids
overlapped_charts = pd.merge(df_codeall[['chart_id']].drop_duplicates(),df_pred1[['chart_id']].drop_duplicates(), how = 'inner')
overlapped_charts = list(overlapped_charts['chart_id'].unique())
print(len(overlapped_charts))  

798


In [29]:
# select the model output
fd_prod = df_pred1[['chart_id','code','start_offset','end_offset','text75','pred']][df_pred1['chart_id'].isin(overlapped_charts)].drop_duplicates()
len(fd_prod)

60131

In [30]:
fd_prod['pred'].value_counts()  # 1 delete

0    34163
1    25968
Name: pred, dtype: int64

In [31]:
# select those in the code output
data = df_codeall[df_codeall['chart_id'].isin(overlapped_charts)]
print(len(data))
print(len(data[data['label'] == 'added']))
print(len(data[data['label'] == 'deleted']))
print(len(data[data['label'] == 'agreed']))

30057
17974
3232
8851


#### The accurary number for the pipeline models

In [32]:
# Use fuzzy logic to compute the deleted, added, agreed
data = data[data['start'] != 0]
data = data[data['end'] != 0]
#data = data[data['created_at'] == date ] ##Change date to filter for datewise data
#label_counts = data.groupby(['label']).size().reset_index(name='counts')
instmatches = data[data['label'] == 'agreed']
instfuzzymatches = data[data['label'] == 'deleted']
instfuzzymatches = instfuzzymatches[instfuzzymatches['deleted_reason'].isin(['Incorrect Specification - Non-Risk Adjusted','repeated_instance','incorrect_year_of_service']) ]
instmatches = pd.concat([instmatches, instfuzzymatches])
instmatches = instmatches[['chart_id','submission_id','code_id','start']].drop_duplicates()

instdeleted = data[data['label'] == 'deleted']
instdeleted = instdeleted[instdeleted['deleted_reason'] != 'other']
instdeleted = pd.merge(instdeleted, instmatches,how = 'outer', on = ['chart_id','submission_id','code_id','start'], indicator = True)
instdeleted = instdeleted[instdeleted['_merge'] == 'left_only']
instdeleted = instdeleted[['chart_id','submission_id','code_id','start','deleted_reason']].drop_duplicates()
deleted_reason = instdeleted.groupby(['deleted_reason']).size().reset_index(name = 'counts')
instdeleted = instdeleted[['chart_id','submission_id','code_id','start']].drop_duplicates()


instadded = data[data['label'] == 'added']
instadded = pd.merge(instadded, instmatches,how = 'outer', on = ['chart_id','submission_id','code_id','start'], indicator = True)
instadded = instadded[instadded['_merge'] == 'left_only']
instadded = instadded[['chart_id','submission_id','code_id','start','end']].drop_duplicates()
instadded = pd.merge(instadded, instdeleted,how = 'outer', on = ['chart_id','submission_id','code_id','start'], indicator = True)
instadded = instadded[instadded['_merge'] == 'left_only']
instadded = instadded[['chart_id','submission_id','code_id','start','end']].drop_duplicates()

added = len(instadded)
deleted = len(instdeleted)
agreed = len(instmatches)
print(added)
print(deleted)
print(agreed)

16098
770
9168


In [33]:
len(fd_prod)

60131

In [34]:
instadded.columns = ['chart_id', 'submission_id', 'code_id', 'start_offset', 'end_offset']
prod_additions_models = pd.merge(instadded, fd_prod,how = 'inner', on = ['chart_id'], indicator = True)
prod_additions_models['abs_diff_start'] = abs(prod_additions_models['start_offset_x'] - prod_additions_models['start_offset_y'])
prod_additions_models['abs_diff_end'] = abs(prod_additions_models['end_offset_x'] - prod_additions_models['end_offset_y'])
prod_additions_models1 = prod_additions_models[ (
                    (prod_additions_models['code_id'] == prod_additions_models['code']) 
                    & (prod_additions_models['abs_diff_start'] <= 75))
          | (prod_additions_models['start_offset_x'] == prod_additions_models['start_offset_y'])
          | (prod_additions_models['end_offset_x'] == prod_additions_models['end_offset_y'])
          |((prod_additions_models['code_id'] == prod_additions_models['code']) 
                    & (prod_additions_models['abs_diff_end'] <= 75))
          ]
instadded_modeldeleted = prod_additions_models1[['chart_id','submission_id','code','start_offset_y','end_offset_y']].drop_duplicates()

In [35]:
len(instadded_modeldeleted)

15128

In [36]:
# initial numbers considering all the additions data for recall

total = len(instmatches) + len(instdeleted) + len(instadded)
added = len(instadded)
deleted = len(instdeleted)
agreed = len(instmatches)

TP = agreed
FN = added
FP = deleted
TN= len(fd_prod)-(len(data)-len(data[data['label'] == 'added']))


precision=TP/(FP+TP)
recall=TP/(FN+TP)
accuracy=(TP+TN)/(TP+TN+FN+FP)

print('TP:',TP)
print('TN:',TN)
print('FN:',FN)
print('FP:',FP)
print('precision:',precision)
print('recall:',recall)
print('accuracy:',accuracy)

print('-------------------')
print('Agreements = ', agreed)
print('Deletions = ', deleted)
print('Additions = ', added)

TP: 9168
TN: 48048
FN: 16098
FP: 770
precision: 0.9225196216542564
recall: 0.3628591783424365
accuracy: 0.7723125101236434
-------------------
Agreements =  9168
Deletions =  770
Additions =  16098


In [37]:
# initial numbers considering model deleted additions data for recall

total = len(instmatches) + len(instdeleted) + len(instadded_modeldeleted)
added = len(instadded_modeldeleted)
deleted = len(instdeleted)
agreed = len(instmatches)

TP = agreed
FN = added
FP = deleted
TN1= len(fd_prod)-(len(data)-len(data[data['label'] == 'added']))

precision=TP/(FP+TP)
recall=TP/(FN+TP)
accuracy=(TP+TN)/(TP+TN+FN+FP)

print('TP:',TP)
print('TN:',TN)
print('FN:',FN)
print('FP:',FP)
print('precision:',precision)
print('recall:',recall)
print('accuracy:',accuracy)

print('-------------------')
print('Agreements = ', agreed)
print('Deletions = ', deleted)
print('Additions = ', added)

TP: 9168
TN: 48048
FN: 15128
FP: 770
precision: 0.9225196216542564
recall: 0.377346065195917
accuracy: 0.7825587438794211
-------------------
Agreements =  9168
Deletions =  770
Additions =  15128


####  Refreshed  Model accuracy numbers

In [38]:
print(len(df_pred1))   # model output with all the instances

76766


In [39]:
print(len(fd_prod))  # already selected  the 798 chart  from previous steps

60131


In [40]:
fd_prod.dtypes

chart_id        object
code            object
start_offset     int64
end_offset       int64
text75          object
pred             int64
dtype: object

In [41]:
TN=len(fd_prod[fd_prod['pred']==1])  # deleted instances but it's not TN true negative
print("TN:",TN)

TN: 25968


In [42]:
# select those 708 charts and only passed the filtering not deleted, 0 is keep here
dev_df_sub_1 = fd_prod[fd_prod['pred']==0]  # select the 798 chart
len(dev_df_sub_1[['chart_id','start_offset']].drop_duplicates())  # 36701 pass through

34163

In [43]:
dev_df_sub_1.rename(columns={'start_offset':'start','end_offset':'end'},inplace=True)

/opt/conda/lib/python3.6/site-packages/pandas/core/frame.py:3781: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


In [44]:
dev_df_sub_1.rename(columns={'code':'code_id'},inplace=True)

In [45]:
new_matches = pd.merge(dev_df_sub_1[['chart_id','start']], instmatches,how = 'inner', on = ['chart_id','start'], indicator = True)
new_deleted = pd.merge(dev_df_sub_1[['chart_id','start']], instdeleted,how = 'inner', on = ['chart_id','start'], indicator = True)

In [46]:
instadded.columns = ['chart_id', 'submission_id', 'code_id', 'start', 'end']
new_added = pd.merge(dev_df_sub_1, instadded,how = 'inner', on = ['chart_id'], indicator = True)
new_added['abs_diff_start'] = abs(new_added['start_x'] - new_added['start_y'])
new_added['abs_diff_end'] = abs(new_added['end_x'] - new_added['end_y'])

In [47]:
new_added1 = new_added[ (
                    (new_added['code_id_x'] == new_added['code_id_y']) 
                    & (new_added['abs_diff_start'] <= 5))
          | (new_added['start_x'] == new_added['start_y'])
          ]

In [48]:
#1. if the start offset matches exactly
#2. If the end offset matches exactly
#3. If the overlap of start offset is between 75 characters and the code id matches
#4. If the overlap of end offset is between 75 characters and the code id matches
new_added2 = new_added[ (
                    (new_added['code_id_x'] == new_added['code_id_y']) 
                    & (new_added['abs_diff_start'] <= 75))
          | (new_added['start_x'] == new_added['start_y'])
          | (new_added['end_x'] == new_added['end_y'])
          |((new_added['code_id_x'] == new_added['code_id_y']) 
                    & (new_added['abs_diff_end'] <= 75))
          ]

In [49]:
instadded_modeldeleted.columns

Index(['chart_id', 'submission_id', 'code', 'start_offset_y', 'end_offset_y'], dtype='object')

In [50]:
#just validating the additions with a different method of calculation
instadded_modeldeleted.columns = ['chart_id', 'submission_id', 'code_id', 'start', 'end']
x = pd.merge(dev_df_sub_1[['chart_id','start']].drop_duplicates()
             , instadded_modeldeleted[['chart_id','start']].drop_duplicates()
             ,how = 'inner', on = ['chart_id','start'], indicator = True)
len(x)

12812

In [51]:
print(len(new_matches))
print(len(new_deleted))
print(len(new_added1[['chart_id','start_x']].drop_duplicates()))
print(len(new_added2[['chart_id','start_x']].drop_duplicates()))

8861
639
9296
12812


In [52]:
#type 1 - older number of additions considering only start offset
refreshed_agreements1 = int(len(new_matches)) + int(len(new_added1[['chart_id','start_x']].drop_duplicates()))
refreshed_deletions1 = int(len(dev_df_sub_1[['chart_id','start']].drop_duplicates())) - (int(len(new_matches)) + int(len(new_added1[['chart_id','start_x']].drop_duplicates()))) 
refreshed_additions1 = added - int(len(new_added1[['chart_id','start_x']].drop_duplicates()))

#type 2 - new number of additions considering start offset and end offset
refreshed_agreements2 = int(len(new_matches)) + int(len(new_added2[['chart_id','start_x']].drop_duplicates()))
refreshed_deletions2 = int(len(dev_df_sub_1[['chart_id','start']].drop_duplicates())) - (int(len(new_matches)) + int(len(new_added2[['chart_id','start_x']].drop_duplicates()))) 
refreshed_additions2 = added - int(len(new_added2[['chart_id','start_x']].drop_duplicates()))

print(refreshed_agreements1)
print(refreshed_deletions1)
print(refreshed_additions1)
print("--------")
print(refreshed_agreements2)
print(refreshed_deletions2)
print(refreshed_additions2)

18157
16006
5832
--------
21673
12490
2316


In [53]:
refreshed_total1 = refreshed_agreements1 + refreshed_deletions1 + refreshed_additions1
refreshed_total2 = refreshed_agreements2 + refreshed_deletions2 + refreshed_additions2
print(refreshed_total1)
print(refreshed_total2)

39995
36479


In [54]:
#type 1 calculation

print('Instance level New Models:')
print('Agreements = ', refreshed_agreements1*100/refreshed_total1,'%')
print('Deletions = ', refreshed_deletions1*100/refreshed_total1,'%')
print('Additions = ', refreshed_additions1*100/refreshed_total1,'%')
print('Precision = ', refreshed_agreements1*100/(refreshed_agreements1+refreshed_deletions1),'%')
print('Recall = ', refreshed_agreements1*100/(refreshed_agreements1+refreshed_additions1),'%')
print('-------------------')
print('Agreements = ', refreshed_agreements1)
print('Deletions = ', refreshed_deletions1)
print('Additions = ', refreshed_additions1)

TP = refreshed_agreements1
FP = refreshed_deletions1
FN = refreshed_additions1

TN=len(fd_prod[fd_prod['pred']==1])-FN # deleted instances

precision=TP/(FP+TP)
recall=TP/(FN+TP)
accuracy=(TP+TN)/(TP+TN+FN+FP)

print('TP:',TP)
print('TN:',TN)
print('FN:',FN)
print('FP:',FP)
print('precision:',precision)
print('recall:',recall)
print('accuracy:',accuracy)


Instance level New Models:
Agreements =  45.39817477184648 %
Deletions =  40.020002500312536 %
Additions =  14.58182272784098 %
Precision =  53.14814272751222 %
Recall =  75.68885739297178 %
-------------------
Agreements =  18157
Deletions =  16006
Additions =  5832
TP: 18157
TN: 20136
FN: 5832
FP: 16006
precision: 0.5314814272751222
recall: 0.7568885739297178
accuracy: 0.6368262626598593


In [55]:
#type 2 calculation

print('Instance level New Models:')
print('Agreements = ', refreshed_agreements2*100/refreshed_total2,'%')
print('Deletions = ', refreshed_deletions2*100/refreshed_total2,'%')
print('Additions = ', refreshed_additions2*100/refreshed_total2,'%')
print('Precision = ', refreshed_agreements2*100/(refreshed_agreements2+refreshed_deletions2),'%')
print('Recall = ', refreshed_agreements2*100/(refreshed_agreements2+refreshed_additions2),'%')
print('-------------------')
print('Agreements = ', refreshed_agreements2)
print('Deletions = ', refreshed_deletions2)
print('Additions = ', refreshed_additions2)



TP = refreshed_agreements2
FP = refreshed_deletions2
FN = refreshed_additions2

TN=len(fd_prod[fd_prod['pred']==1])-FN  # deleted instances

precision=TP/(FP+TP)
recall=TP/(FN+TP)
accuracy=(TP+TN)/(TP+TN+FN+FP)

print('TP:',TP)
print('TN:',TN)
print('FN:',FN)
print('FP:',FP)
print('precision:',precision)
print('recall:',recall)
print('accuracy:',accuracy)



Instance level New Models:
Agreements =  59.41226459058637 %
Deletions =  34.23887716220291 %
Additions =  6.348858247210724 %
Precision =  63.43997892456751 %
Recall =  90.34557505523365 %
-------------------
Agreements =  21673
Deletions =  12490
Additions =  2316
TP: 21673
TN: 23652
FN: 2316
FP: 12490
precision: 0.6343997892456752
recall: 0.9034557505523365
accuracy: 0.7537709334619415
